In [ ]:
# !pip install pybind11; 
# !git clone --branch develop https://github.com/IAZero/wisardpkg.git; 
# !cd wisardpkg; python setup.py install;

In [1]:
# !pip install wisardpkg==2.0.0a7

In [2]:
import wisardpkg as wp

import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [3]:
def encode_var(var, borders, bins=10):
  min, max = borders
  k = (max - min)/bins
  a = var - min
  return np.array([1 if i <= a//k else 0 for i in range(bins)])

def encode(x, ranges, bins=20):
  data = [encode_var(v, b, bins=bins) for v, b in zip(x, ranges)]
  return wp.BinInput(flatten(data))

def flatten(var):
  return np.reshape(var, (-1,)).tolist()

def predict(model, X):
  y_predict = []
  for x  in X:
    y_predict.append(model.predict(x))
  return y_predict

In [4]:
info = fetch_california_housing()
X, y = info['data'], info['target']

In [5]:
ranges = []
for i in range(X.shape[1]):
  ranges.append((np.min(X[:,i]), np.max(X[:,i])))
ranges

[(0.4999, 15.0001),
 (1.0, 52.0),
 (0.8461538461538461, 141.9090909090909),
 (0.3333333333333333, 34.06666666666667),
 (3.0, 35682.0),
 (0.6923076923076923, 1243.3333333333333),
 (32.54, 41.95),
 (-124.35, -114.31)]

In [6]:
X_bin = [encode(x, ranges, bins=20) for x in X]
X_train, X_test, y_train, y_test = train_test_split(X_bin, y, test_size=0.3, random_state=1)

In [7]:
model = wp.RegressionWisard(10, minOnes=2)

AttributeError: module 'wisardpkg' has no attribute 'RegressionWisard'

In [ ]:
for x, y in zip(X_train, y_train):
  model.train(x, y) 

In [ ]:
y_predict = predict(model, X_train)

In [ ]:
metrics.mean_absolute_error(y_train, y_predict), metrics.mean_squared_error(y_train, y_predict), metrics.r2_score(y_train, y_predict)

In [ ]:
y_predict = predict(model, X_test)

In [ ]:
metrics.mean_absolute_error(y_test, y_predict), metrics.mean_squared_error(y_test, y_predict), metrics.r2_score(y_test, y_predict)

In [ ]:
mae_train, mse_train, r2_train = [], [], []
mae_test, mse_test, r2_test = [], [], []


for i in range(2, len(X_bin[0])):
  model = wp.RegressionWisard(i, minOnes=2)

  for x, y in zip(X_train, y_train):
    model.train(x, y) 
  
  y_predict = predict(model, X_train)

  mae_train.append(metrics.mean_absolute_error(y_train, y_predict))
  mse_train.append(metrics.mean_squared_error(y_train, y_predict)) 
  r2_train.append(metrics.r2_score(y_train, y_predict))

  y_predict = predict(model, X_test)

  mae_test.append(metrics.mean_absolute_error(y_test, y_predict))
  mse_test.append(metrics.mean_squared_error(y_test, y_predict)) 
  r2_test.append(metrics.r2_score(y_test, y_predict))

plt.plot(mae_test, label='MAE')
plt.plot(mse_test, label='MSE')
plt.plot(r2_test, label='R2')

plt.legend()